In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from signals import *
from frequencyestimator import *
from scipy.optimize import basinhopping, minimize
from tqdm.auto import tqdm

sns.set_style("whitegrid")
sns.despine(left=True, bottom=True)
sns.set_context("poster", font_scale = .45, rc={"grid.linewidth": 0.8})

<Figure size 640x480 with 0 Axes>

In [3]:
P0 = lambda n, theta: np.cos((2*n+1)*theta)**2
P1 = lambda n, theta: np.sin((2*n+1)*theta)**2

def estimate_signal(depths, n_samples, theta):
        signals = np.zeros(len(depths), dtype = np.complex128)
        measurements = np.zeros(len(depths))
        for i,n in enumerate(depths):
            # Get the exact measuremnt probabilities
            p0 = P0(n, theta)
            p1 = P1(n, theta)

            p0x = P0x(n,theta)
            p1x = P1x(n,theta)
            
            p0_estimate = np.random.binomial(n_samples[i], p0)/n_samples[i]
            p1_estimate = 1 - p0_estimate

            p0x_estimate = np.random.binomial(n_samples[i], p0x)/n_samples[i]
            p1x_estimate = 1.0 - p0x_estimate
            measurements[i] = p0_estimate
            
            # Estimate theta
            theta_estimated = np.arctan2(p0x_estimate - p1x_estimate, p0_estimate - p1_estimate)

            # Compute f(n) - Eq. 3
            fi_estimate = np.exp(1.0j*theta_estimated)
            signals[i] = fi_estimate
         
        return signals, measurements

from scipy.stats import binom

def apply_correction(ula_signal, measurements, theta_est, theta):
    theta_est = np.abs(theta_est)
    p_exact = np.cos((2*ula_signal.depths+1)*(theta))**2
    p_neg = np.cos((2*ula_signal.depths+1)*(-theta))**2
    p_o2 = np.cos((2 * ula_signal.depths + 1) * (theta_est/2.0)) ** 2
    p_o4 = np.cos((2 * ula_signal.depths + 1) * (theta_est/4.0)) ** 2
    p_same = np.cos((2*ula_signal.depths+1)*(theta_est))**2
    p_s2 = np.cos((2 * ula_signal.depths + 1) * (np.pi/2-theta_est)) ** 2
    p_s4 = np.cos((2 * ula_signal.depths + 1) * (np.pi / 4 - theta_est)) ** 2
    p_s2_o2 = np.cos((2 * ula_signal.depths + 1) * (np.pi / 2 - theta_est/2)) ** 2
    p_s4_o2 = np.cos((2 * ula_signal.depths + 1) * (np.pi / 4 - theta_est/2)) ** 2

    l_exact = np.sum(
        np.log([1e-75 + binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk],
                                  p_exact[kk]) for kk in
                range(len(ula_signal.n_samples))]))
    l_neg = np.sum(
        np.log([1e-75 + binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk],
                                  p_neg[kk]) for kk in
                range(len(ula_signal.n_samples))]))
    l_o2 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk]*measurements[kk], ula_signal.n_samples[kk], p_o2[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_o4 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_o4[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_same = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_same[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_s2 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_s2[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_s4 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_s4[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_s2_o2 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_s2_o2[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_s4_o2 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_s4_o2[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    
    which_correction = np.argmax([l_same, l_s2, l_s4, l_o2, l_o4, l_s2_o2, l_s4_o2])
    if which_correction == 1:
        theta_est = np.pi/2.0 - theta_est
    elif which_correction == 2:
        theta_est = np.pi/4.0 - theta_est
    elif which_correction == 3:
        theta_est = theta_est/2
    elif which_correction == 4:
        theta_est = theta_est/4
    elif which_correction == 5:
        theta_est = np.pi / 2.0 - 0.5 * theta_est
    elif which_correction == 6:
        theta_est = np.pi / 4.0 - 0.5 * theta_est
    # elif which_correction == 7:
    #     theta_est = -theta_est

    # print(f'FINAL ANGLE FOUND: {theta_est, theta}')

    return np.abs(theta_est)

In [4]:
import numpy as np
import itertools
from typing import List

def generate_all_sign_variations(signs_array: np.ndarray) -> np.ndarray:
    """
    Generate all possible sign variations by changing signs at all possible pairs of positions.
    
    Parameters:
    -----------
    signs_array : numpy.ndarray
        Original array of signs (typically containing 1 or -1)
    
    Returns:
    --------
    numpy.ndarray
        Array of all possible sign variation arrays
    """
    # Get all possible unique pairs of positions
    n = len(signs_array)
    position_pairs = list(itertools.combinations(range(n), 2))
    
    # Will store all variations
    all_variations = []
    
    # Iterate through all possible position pairs
    for pos1, pos2 in position_pairs:
        # Generate variations for this pair of positions
        pair_variations = generate_pair_variations(signs_array, pos1, pos2)
        all_variations.extend(pair_variations)
    
    return all_variations

def generate_adjacent_sign_variations(signs_array: np.ndarray, size: int) -> np.ndarray:
    """
    Generate sign variations by sliding a two-position window across the array.
    
    Parameters:
    -----------
    signs_array : numpy.ndarray
        Original array of signs (typically containing 1 or -1)
    
    Returns:
    --------
    numpy.ndarray
        Array of all possible sign variation arrays
    """
    # Total length of the array
    n = len(signs_array)
    
    # Will store all variations
    all_variations = []
    
    # Iterate through adjacent position pairs 
    # (0,1), (1,2), (2,3), ... until the second-to-last pair
    for pos1 in range(1, n - size + 1):
        pos = [pos1 + i for i in range(size)]
        
        # Generate variations for this pair of adjacent positions
        pair_variations = generate_pair_variations(signs_array, pos)
 
        all_variations.extend(pair_variations)
    
    return all_variations

def generate_pair_variations(signs_array: np.ndarray, pos: List[int]) -> List[np.ndarray]:
    """
    Generate sign variations for a specific pair of positions.
    
    Parameters:
    -----------
    signs_array : numpy.ndarray
        Original array of signs
    pos1 : int
        First position to modify
    pos2 : int
        Second position to modify
    
    Returns:
    --------
    List[numpy.ndarray]
        List of sign variation arrays
    """
    # Validate input positions
    # if pos1 < 0 or pos2 < 0 or pos1 >= len(signs_array) or pos2 >= len(signs_array):
    #     raise ValueError("Positions must be within the array bounds")
    
    # Generate all possible sign combinations for the two positions
    sign_combinations = list(itertools.product([-1, 1], repeat=len(pos)))
    
    # Create variations
    variations = []
    for combo in sign_combinations:
        # Create a copy of the original array
        variation = signs_array.copy()
        
        # Modify the two specified positions
        for i in range(len(combo)):
            variation[pos[i]] *= combo[i]
        
        variations.append(tuple(variation))
    
    return variations


# Example array
signs = np.array([1]*4)

# Generate variations for all position pairs
all_variations = generate_adjacent_sign_variations(signs, 3)

# print("Original array:", signs)
# print(f"\nTotal variations: {len(all_variations)}")
# print(f'4*len(signs)^2: {4*len(signs)**2}')
# print(f'2^len(signs): {2**len(signs)}')

print("\nFirst few variations:")
for var in all_variations:  # Print first 10 variations
    print(var)


First few variations:
(1, -1, -1, -1)
(1, -1, -1, 1)
(1, -1, 1, -1)
(1, -1, 1, 1)
(1, 1, -1, -1)
(1, 1, -1, 1)
(1, 1, 1, -1)
(1, 1, 1, 1)


## Define the objective function to minimize

In [5]:
def objective_function_eig(lp, cos_signal, abs_sin, ula_signal, esprit):   
    signal = cos_signal + 1.0j * lp * abs_sin
    R = ula_signal.get_cov_matrix_toeplitz(signal)
    _, _ = esprit.estimate_theta_toeplitz(R)
    eigs = np.abs(esprit.eigs)[:2]
    obj = eigs[1] - eigs[0]

    return obj

def objective_function(lp, cos_signal, abs_sin, ula_signal, esprit):
    signal = cos_signal + 1.0j * lp * abs_sin
    R = ula_signal.get_cov_matrix_toeplitz(signal)
    theta_est, _ = esprit.estimate_theta_toeplitz(R)

    theta_est = apply_correction(ula_signal, ula_signal.measurements, theta_est, theta_est)

    # print(f'2*theta_est: {2*theta_est}')
    p_same = np.cos((2 * ula_signal.depths + 1) * (theta_est)) ** 2

    obj = -np.sum(
        np.log(
            [1e-75 + binom.pmf(ula_signal.n_samples[kk] * ula_signal.measurements[kk], ula_signal.n_samples[kk], p_same[kk]) for kk
             in
             range(len(ula_signal.n_samples))]))


    # eigs = np.abs(esprit.eigs)[:2]
    # obj = eigs[1] - eigs[0]

    return obj

## What is the distribution of the signs? 

In [6]:
narray = [2,2,2,2,2,3]

def get_heavy_signs(narray, steps):

    thetas = np.linspace(np.arcsin(0.09), np.arcsin(0.91), steps)
    avals = np.sin(thetas)

    ula_signal = TwoqULASignal(M=narray, C=3)

    num_mc = 1000

    sign_distributions = {}

    for a,theta in zip(avals,thetas):
        # theta = np.arcsin(a)
        distr = {}
        for i in range(num_mc):
            signal, _ = estimate_signal(ula_signal.depths, ula_signal.n_samples, theta)
                # print(measurements)
            signs = tuple(np.sign(np.imag(signal)))
            # print(signs)

            distr[signs] = distr.get(signs, 0.0) + 1/num_mc
        sign_distributions[str(a)] = distr

    def get_signs(sign_distribution):
        # Sort the dictionary by values in descending order
        sorted_data = dict(sorted(sign_distribution.items(), key=lambda item: item[1], reverse=True))

        # Create a new dictionary with cumulative sum close to 0.68
        cumulative_dict = {}
        cumulative_sum = 0.0

        for key, value in sorted_data.items():
            cumulative_dict[key] = value
            if cumulative_sum + value > 0.9:
                break
            cumulative_sum += value

        # Output the result
        return cumulative_dict
    
    ret_dict = {}
    for a in avals:
        distr = get_signs(sign_distributions[str(a)])
        ret_dict[str(a)] = distr
 
    # Normalize the values
    normalized_data = {}
    for key, sub_dict in ret_dict.items():
        total_sum = sum(sub_dict.values())
        normalized_data[key] = {k: v / total_sum for k, v in sub_dict.items()}

    return normalized_data

normalized_data = get_heavy_signs(narray, 10)

# normalized_data

In [7]:
import ast
# Taking a random sample based on the normalized probabilities
def sample_from_normalized(data, sample_size=1):
    keys = list(map(str, list(data.keys())))
    probabilities = list(data.values())
    sampled_keys = np.random.choice(a=keys, size=sample_size, p=probabilities)
    return [ast.literal_eval(t) for t in sampled_keys]
avals = list(normalized_data.keys())
# Get one random sample
sampled = sample_from_normalized(normalized_data[avals[0]], sample_size=3)
# print("Sampled keys:",sampled)

## Use the distribution of signs so we don't have to try all possible signs

In [8]:
np.random.seed(42)

a = 0.1
print(a)
theta = np.arcsin(a)

narray = [2,2,2,2,2,2,2,2]
heavy_signs = get_heavy_signs(narray, len(narray)**2)

ula_signal = TwoqULASignal(M=narray, C=5)
esprit = ESPIRIT()

def sample_signs(heavy_signs, sample_size=3):
    """
    Sample a set of sign variations from a learned sign distribution.

    Args:
        heavy_signs (dict): A dictionary where the keys are strings representing float values,
                           and the values are dictionaries with keys as sign vectors and values
                           as their corresponding probabilities.
        sample_size (int, optional): The number of sign variations to sample. Defaults to 3.

    Returns:
        dict: A dictionary where the keys are the same as the keys in `heavy_signs`,
              and the values are lists of sampled sign variations.
    """
    avals = list(heavy_signs.keys())
    signs_to_try = {}
    for a in avals:
        signs_to_try[a] = list(set(sample_from_normalized(heavy_signs[a], sample_size=sample_size)))

    return signs_to_try

def avals_to_usef(a0, avals, L=3):
    """
    Find the `L` values in `avals` that are closest to the given `a0` value.

    Args:
        a0 (float): The reference value to find the closest `L` values for.
        avals (list): A list of float values representing the keys in `heavy_signs`.
        L (int, optional): The number of closest values to return. Defaults to 3.

    Returns:
        list: A list of string representations of the `L` closest values to `a0` in `avals`.
    """
    avals = list(map(float, avals))

    left = 0
    right = len(avals)
    
    while left < right:
        mid = (left + right) // 2
        
        if avals[mid] <= a0:
            left = mid + 1
        else:
            right = mid

    if left >= 4:
        avals_to_use = list(map(str, avals[left-L: left+L]))
    if left >= 3:
        avals_to_use = list(map(str, avals[left-3: left+L]))
    elif left>=2:
        avals_to_use = list(map(str, avals[left-2: left+L]))
    elif left>=1:
        avals_to_use = list(map(str, avals[left-1: left+L]))
    else:
        avals_to_use = list(map(str, avals[left: left+L]))
    
    return avals_to_use

def all_signs_to_try(avals_to_use, signs_to_try, adjacency=2):
    """
    Generate all possible sign variations within a Hamming distance of 2 from the given `avals_to_use`.

    Args:
        avals_to_use (list): A list of string representations of float values.
        signs_to_try (dict): A dictionary where the keys are string representations of float values,
                            and the values are lists of sign variations.
        adjacency (int, optional): The maximum Hamming distance to consider for the sign variations.
                                  Defaults to 2.

    Returns:
        list: A list of all possible sign variations within the specified Hamming distance.
    """
    all_signs = []
    for a in avals_to_use:
        for x in signs_to_try[a]:
            hamming_distance_two_signs = generate_adjacent_sign_variations(np.array(x), adjacency)
            all_signs.extend(hamming_distance_two_signs)
    all_signs = list(set(all_signs))
    
    return all_signs

def minimize_obj(all_signs, cos_signal, abs_sin, ula_signal, esprit, disp):
    """
    Find the sign variation that minimizes the objective function.

    Args:
        all_signs (list): A list of all possible sign variations to consider.
        cos_signal (numpy.ndarray): The real part of the estimated signal.
        abs_sin (numpy.ndarray): The absolute value of the imaginary part of the estimated signal.
        ula_signal (ULASignal): An instance of the ULASignal class, containing information about the signal.
        esprit (ESPRIT): An instance of the ESPRIT class, used for estimating the signal parameters.
        disp (bool): If True, prints the current objective and the current best sign variation.

    Returns:
        numpy.ndarray: The sign variation that minimizes the objective function.
    """
    # obj = 1.0
    x_star = np.array(all_signs[0])
    obj = objective_function(x_star, cos_signal, abs_sin, ula_signal, esprit)
    for x in all_signs:
        curr_obj = objective_function(np.array(x), cos_signal, abs_sin, ula_signal, esprit)
        if curr_obj < obj:
            if disp:
                print(f'current objective: {curr_obj}')
                print(f'current best signs: {x}')
            obj = curr_obj
            x_star = np.array(x)

    return x_star

def csae_with_local_minimization(theta, ula_signal, esprit, heavy_signs, sample=False, correction=False, optimize=False, disp=False):
    """
    Perform CSAE (Compressive Sensing Angle Estimation) with local minimization.

    Args:
        theta (float): The true angle of the signal.
        ula_signal (ULASignal): An instance of the ULASignal class, containing information about the signal.
        esprit (ESPRIT): An instance of the ESPRIT class, used for estimating the signal parameters.
        heavy_signs (dict): A dictionary where the keys are strings representing float values,
                           and the values are dictionaries with keys as sign vectors and values
                           as their corresponding probabilities.
        sample (bool, optional): If True, samples sign variations from the learned sign distribution.
                                If False, uses the learned sign distribution directly. Defaults to False.
        correction (bool, optional): If True, applies a correction to the estimated angle. Defaults to False.
        optimize (bool, optional): If True, uses a sliding window approach to find the best sign variation.
                                  If False, uses the learned sign distribution directly. Defaults to False.
        disp (bool, optional): If True, prints additional information during the process. Defaults to False.

    Returns:
        dict: A dictionary containing the estimated angles, errors, number of queries, maximum depth,
              and other relevant information.
    """

    depths = ula_signal.depths
    n_samples = ula_signal.n_samples
    csignal, measurements = estimate_signal(depths, n_samples, theta)
    ula_signal.measurements = measurements

    cos_signal = np.real(csignal)

    correct_signs = np.sign(np.imag(csignal))
    if disp:
        print(f'correct signs: {correct_signs}')
    abs_sin = np.abs(np.imag(csignal))

    if sample:
        # step 1: sample signs from learned sign distribution
        signs_to_try = sample_signs(heavy_signs=heavy_signs, sample_size=3)
        avals = list(heavy_signs.keys())

        if optimize:
            # step 2: using rough estimate where the amplitude is, use the a-values around that estimate
            a0 = np.sqrt(0.5 - 0.5 * cos_signal[0])
            avals_to_use = avals_to_usef(a0, avals, L=3)

            if disp:
                print(f'rough estimate a: {a0}')
                print(f'avals to use: {avals_to_use}')

            # step 3: now vary the signs in a sliding window of size "adjacency"
            all_signs = all_signs_to_try(avals_to_use, signs_to_try, adjacency=2)

            if disp:
                print(f'number of signs Hamming distance two: {len(all_signs)}')

            # step 4: try all the signs pick the ones that minimize the objective function
            if disp:
                print('debug')
                print(all_signs)
            x_star = minimize_obj(all_signs, cos_signal, abs_sin, ula_signal, esprit, disp)

            # step 5 (optional): do one more sweep
            hamming_distance_one_signs = list(set(generate_adjacent_sign_variations(x_star, 1)))
            x_star = minimize_obj(hamming_distance_one_signs, cos_signal, abs_sin, ula_signal, esprit, disp)
        
        else:
            # here we don't vary the signs using a sliding window, but directly use the learned sign distribution (poor performance)
            all_signs = []
            for a in avals:
                all_signs.extend(heavy_signs[a])
            all_signs = list(set(all_signs))
            x_star = minimize_obj(all_signs, cos_signal, abs_sin, ula_signal, esprit, disp)

    else:
        avals = list(heavy_signs.keys())
        signs_to_try = {}
        for a in avals:
            signs_to_try[a] = heavy_signs[a]

        if optimize:
            # step 2: using rough estimate where the amplitude is, use the a-values around that estimate
            a0 = np.sqrt(0.5 - 0.5 * cos_signal[0])
            avals_to_use = avals_to_usef(a0, avals, L=3)

            if disp:
                print(f'rough estimate a: {a0}')
                print(f'avals to use: {avals_to_use}')

            # step 3: now vary the signs in a sliding window of size "adjacency"
            all_signs = all_signs_to_try(avals_to_use, signs_to_try, adjacency=2)

            if disp:
                print(f'number of signs Hamming distance two: {len(all_signs)}')

            # step 4: try all the signs pick the ones that minimize the objective function
            x_star = minimize_obj(all_signs, cos_signal, abs_sin, ula_signal, esprit, disp, measu)

            # step 5 (optional): do one more sweep
            hamming_distance_one_signs = list(set(generate_adjacent_sign_variations(x_star, 1)))
            x_star = minimize_obj(hamming_distance_one_signs, cos_signal, abs_sin, ula_signal, esprit, disp)
        
        else:
            # here we don't vary the signs using a sliding window, but directly use the learned sign distribution (poor performance)
            all_signs = []
            for a in avals:
                all_signs.extend(heavy_signs[a])
            all_signs = list(set(all_signs))
            x_star = minimize_obj(all_signs, cos_signal, abs_sin, ula_signal, esprit, disp)


    # Optimization is done.

    if disp:
        print(x_star)
        
    signal = cos_signal + 1.0j * x_star * abs_sin
    R = ula_signal.get_cov_matrix_toeplitz(signal)
    theta_est, _ = esprit.estimate_theta_toeplitz(R)
    theta_est = np.abs(theta_est)
    eigs = np.abs(esprit.eigs)[:2]
    basin_obj = eigs[1] - eigs[0]
    if correction:
        theta_est = apply_correction(ula_signal, measurements, theta_est, theta) #apply correction
    if disp:
        print(f'basin obj: {basin_obj}')

    cR = ula_signal.get_cov_matrix_toeplitz(csignal)
    theta_est1, _ = esprit.estimate_theta_toeplitz(cR)
    eigs = np.abs(esprit.eigs)[:2]
    true_obj = eigs[1] - eigs[0]
    if disp:
        print(f'true obj: {true_obj}')

    # compute queries required
    num_queries = np.sum(np.array(ula_signal.depths)*np.array(ula_signal.n_samples)) + ula_signal.n_samples[0]
    max_single_query = np.max(ula_signal.depths)

    ret_dict = {'theta_est': theta_est, 'theta_est1': theta_est1, 
                'error': np.abs(np.sin(theta)-np.sin(theta_est)), 
                'error1': np.abs(np.sin(theta)-np.sin(theta_est1)), 
                'queries': num_queries, 'depth': max_single_query,
                'true_obj': true_obj, 'basin_obj': basin_obj,
                'x_star': x_star}
    
    return ret_dict

csae_with_local_minimization(theta, ula_signal, esprit, heavy_signs, sample=True, correction=True, optimize=True, disp=True)

0.1
correct signs: [ 1.  1.  1.  1. -1.  1.  1.  1.  1.]
rough estimate a: 0.18910752115495127
avals to use: ['0.13981296510949756', '0.15634532387578837', '0.1728339924311597', '0.18927436306884374', '0.20566184157877418', '0.22199184853142423']
number of signs Hamming distance two: 114
debug
[(1.0, -1.0, -1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0), (1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0), (1.0, 1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0, -1.0), (1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0), (1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0), (1.0, 1.0, 1.0, 1.0, -1.0, -1.0, 1.0, -1.0, -1.0), (1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0), (1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, 1.0, 1.0), (1.0, 1.0, -1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0), (1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0), (1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 1.0, -1.0), (1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, -1.0), (1.0, 1.0, 1.0, 1.0, -1.0, 0.0, 1.0, -1.0, -1.0), (1.0, 1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0),

{'theta_est': 0.10014050463580297,
 'theta_est1': 0.10012026433053867,
 'error': 2.678164120033244e-05,
 'error1': 4.692056561073077e-05,
 'queries': 2600,
 'depth': 128,
 'true_obj': -397.4516598486232,
 'basin_obj': -71.49676886330407,
 'x_star': array([ 1., -1.,  1.,  1.,  1., -1.,  1., -1.,  1.])}

## Let's do some statistics now

In [9]:
a0=0.14658716786698373
signs_to_try = sample_signs(heavy_signs=heavy_signs, sample_size=3)
avals = list(heavy_signs.keys())
avals_to_usef(a0, avals, L=3), avals, a0

(['0.10663566752567158',
  '0.12324153604814797',
  '0.13981296510949756',
  '0.15634532387578837',
  '0.1728339924311597',
  '0.18927436306884374'],
 ['0.09',
  '0.10663566752567158',
  '0.12324153604814797',
  '0.13981296510949756',
  '0.15634532387578837',
  '0.1728339924311597',
  '0.18927436306884374',
  '0.20566184157877418',
  '0.22199184853142423',
  '0.23825982055751374',
  '0.2544612116232283',
  '0.2705914943005945',
  '0.286646161032655',
  '0.3026207253930916',
  '0.318510723339943',
  '0.33431171446306707',
  '0.35001928322499964',
  '0.3656290401948629',
  '0.3811366232749777',
  '0.3965376989198384',
  '0.41182796334710836',
  '0.42700314374029846',
  '0.44205899944279253',
  '0.4569913231428852',
  '0.47179594204950215',
  '0.48646871905827366',
  '0.5010055539076351',
  '0.5154023843246325',
  '0.5296551871601116',
  '0.5437599795129751',
  '0.5577128198431908',
  '0.5715098090732432',
  '0.5851470916777191',
  '0.5986208567607206',
  '0.6119273391208092',
  '0.625062

In [84]:
np.random.seed(42)

avals = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
# avals = [np.random.uniform(0.1, 0.9)]
# avals = [0.5, 0.6, 0.7, 0.8, 0.9]
# avals = [0.1, 0.2, 0.3]
avals = [np.random.uniform(0.1, 0.9) for _ in range(15)]
# narray = [2]*5 + [3]
narray = [2]*12

print('learning the distribution of signs...')
heavy_signs = get_heavy_signs(narray, int(6*len(narray)))
num_queries = np.zeros(len(avals), dtype=int)
max_single_query = np.zeros(len(avals), dtype=int)

num_mc=100
thetas = np.zeros((len(avals), num_mc))
errors = np.zeros((len(avals), num_mc))
thetas1 = np.zeros((len(avals), num_mc))
errors1 = np.zeros((len(avals), num_mc))

basin_obj = np.zeros((len(avals), num_mc))
true_obj = np.zeros((len(avals), num_mc))

ula_signal = TwoqULASignal(M=narray, C=5)
esprit = ESPIRIT()

for j,a in enumerate(avals):
    theta = np.arcsin(a)
    print(f'theta = {theta}')
    disp=False
    # if j==4:
    #     disp=True
    for i in tqdm(range(num_mc)):
        res = csae_with_local_minimization(theta, ula_signal, esprit, heavy_signs, sample=True, correction=True, optimize=True, disp=disp)
        thetas[j][i] = res['theta_est']
        errors[j][i] = res['error']

        thetas1[j][i] = res['theta_est1']
        errors1[j][i] = res['error1']

        basin_obj[j][i] = res['basin_obj']
        true_obj[j][i] = res['true_obj']
    num_queries[j] = res['queries']
    max_single_query[j] = res['depth']

    print(f'constant factors query and depth: {np.percentile(errors[j], 95):.3e}, {np.percentile(errors[j], 95) * num_queries[j]}, {np.percentile(errors[j], 95) * max_single_query[j]}')

learning the distribution of signs...
theta = 0.41111546403370536


  0%|          | 0/100 [00:00<?, ?it/s]

constant factors query and depth: 1.580e-04, 6.48000014142134, 0.3235260918973892
theta = 1.0363905664439828


  0%|          | 0/100 [00:00<?, ?it/s]

constant factors query and depth: 6.514e-05, 2.672115510222025, 0.13341035019343508
theta = 0.7554209219922302


...

constant factors query and depth: 8.372e-05, 3.434062125310412, 0.17145195593943746
theta = 0.2732593578259982


  0%|          | 0/100 [00:00<?, ?it/s]

constant factors query and depth: 1.144e-04, 4.69192745406286, 0.23425322832571274
theta = 0.24799415401854524


  0%|          | 0/100 [00:00<?, ?it/s]

constant factors query and depth: 1.561e-04, 6.401369703102888, 0.3196003206229818


In [85]:
for i in range(len(avals)):
    print(np.sum(np.where(np.array(basin_obj[i])/np.array(true_obj[i]) >= 0.99, 1, 0))/len(basin_obj[i]))

0.0
0.0
0.01
0.22
0.0
0.01
0.01
0.02
0.27
0.01
0.0
0.28
0.02
0.0
0.07


In [86]:
i=6
np.array(basin_obj[i])/np.array(true_obj[i]), np.sum(np.where(np.array(basin_obj[i])/np.array(true_obj[i]) >= 0.99, 1, 0))/len(basin_obj[i])

(array([0.60287013, 0.62426138, 0.1552    , 0.12820457, 0.74984684,
        0.11309817, 0.10205856, 0.18151208, 0.01025888, 1.07665288,
        0.65926606, 0.64222144, 0.52598446, 0.03310375, 0.09635233,
        0.06447469, 0.74235025, 0.53985041, 0.14894406, 0.01606382,
        0.0860895 , 0.41686863, 0.4290147 , 0.06819898, 0.36073965,
        0.22815559, 0.8533285 , 0.13185352, 0.15874073, 0.33209937,
        0.03474256, 0.11140144, 0.22300462, 0.14127383, 0.53978022,
        0.25635188, 0.24930173, 0.0903205 , 0.15455531, 0.24004338,
        0.33376168, 0.37508087, 0.02197168, 0.10819385, 0.66944568,
        0.8684042 , 0.01282616, 0.19967174, 0.81841865, 0.72881081,
        0.2672098 , 0.02357927, 0.0478728 , 0.23587034, 0.00491157,
        0.40701626, 0.83853877, 0.74392893, 0.6445632 , 0.69775913,
        0.42904069, 0.21652277, 0.12389435, 0.0146493 , 0.1662096 ,
        0.88416806, 0.09640114, 0.32454476, 0.7199318 , 0.19599446,
        0.34638071, 0.26790821, 0.08166197, 0.95

In [87]:
i=6
np.sin(thetas[i]), avals[i]

(array([0.14641039, 0.14640941, 0.14644366, 0.14655684, 0.14641173,
        0.14643392, 0.14643114, 0.1464998 , 0.14646828, 0.1462315 ,
        0.1463911 , 0.14648452, 0.14647165, 0.14643394, 0.1464613 ,
        0.14652702, 0.14639447, 0.146474  , 0.14648108, 0.14644696,
        0.14643249, 0.14654521, 0.14646636, 0.14642111, 0.14648187,
        0.14648035, 0.14643905, 0.146431  , 0.14646749, 0.14642712,
        0.14642209, 0.146406  , 0.1468935 , 0.14652148, 0.14644381,
        0.14646965, 0.14647083, 0.14657738, 0.14640695, 0.14647109,
        0.14648093, 0.14648799, 0.14658424, 0.14640409, 0.14651846,
        0.14648327, 0.14646388, 0.14644556, 0.1466155 , 0.14639432,
        0.14646256, 0.14640862, 0.14648253, 0.14647328, 0.14644428,
        0.14644462, 0.14649361, 0.14644277, 0.14619638, 0.14644603,
        0.1464861 , 0.14641326, 0.14641476, 0.14639815, 0.14643764,
        0.14643742, 0.14647562, 0.14647285, 0.14644206, 0.14646506,
        0.14650428, 0.14648803, 0.14657271, 0.14

In [88]:
i=6
thetas[i]/np.pi, np.arcsin(avals[i])/np.pi

(array([0.046772  , 0.04677168, 0.04678271, 0.04681913, 0.04677243,
        0.04677957, 0.04677868, 0.04680077, 0.04679063, 0.04671444,
        0.04676579, 0.04679585, 0.04679171, 0.04677958, 0.04678838,
        0.04680953, 0.04676688, 0.04679247, 0.04679475, 0.04678377,
        0.04677911, 0.04681538, 0.04679001, 0.04677545, 0.046795  ,
        0.04679451, 0.04678122, 0.04677863, 0.04679038, 0.04677738,
        0.04677577, 0.04677059, 0.04692746, 0.04680775, 0.04678276,
        0.04679107, 0.04679145, 0.04682574, 0.0467709 , 0.04679153,
        0.0467947 , 0.04679697, 0.04682794, 0.04676997, 0.04680678,
        0.04679545, 0.04678921, 0.04678332, 0.046838  , 0.04676683,
        0.04678879, 0.04677143, 0.04679521, 0.04679224, 0.04678291,
        0.04678302, 0.04679878, 0.04678242, 0.04670314, 0.04678347,
        0.04679637, 0.04677292, 0.04677341, 0.04676806, 0.04678077,
        0.0467807 , 0.04679299, 0.0467921 , 0.04678219, 0.04678959,
        0.04680221, 0.04679699, 0.04682423, 0.04

In [89]:
# np.random.seed(1)

# csignal, measurements = estimate_signal(ula_signal.depths, ula_signal.n_samples, np.arcsin(avals[i]))

# plt.plot(measurements, 'r', label='measurements')
# plt.plot(np.cos((2*ula_signal.depths + 1)*theta2)**2, 'b', label='theta2')
# plt.plot(np.cos((2*ula_signal.depths + 1)*theta1)**2, 'k', label='theta1')
# plt.legend()

In [90]:
perc = 68
avg = 0.0
avg_err = 0.0
for i in range(len(avals)):
    avg += np.percentile(errors[i], perc) * num_queries[i]/len(avals)
    avg_err += np.percentile(errors[i], perc)/len(avals)
    print(f'constant factors query and depth ({perc}%) for {avals[i]:.3f}: {np.percentile(errors[i], perc):.3e}, {np.percentile(errors[i], perc) * num_queries[i]:.3f}, {np.percentile(errors[i], perc) * max_single_query[i]:.3e}')
print(f'average constant factor {avg:0.3f}')
print(f'average error {avg_err:0.3e}')

constant factors query and depth (68%) for 0.400: 4.264e-05, 1.749, 8.733e-02
constant factors query and depth (68%) for 0.861: 2.031e-05, 0.833, 4.160e-02
constant factors query and depth (68%) for 0.686: 3.415e-05, 1.401, 6.994e-02
constant factors query and depth (68%) for 0.579: 3.665e-05, 1.503, 7.506e-02
constant factors query and depth (68%) for 0.225: 4.541e-05, 1.863, 9.300e-02
constant factors query and depth (68%) for 0.225: 4.742e-05, 1.945, 9.712e-02
constant factors query and depth (68%) for 0.146: 5.491e-05, 2.252, 1.124e-01
constant factors query and depth (68%) for 0.793: 2.637e-05, 1.082, 5.401e-02
constant factors query and depth (68%) for 0.581: 3.474e-05, 1.425, 7.114e-02
constant factors query and depth (68%) for 0.666: 4.285e-05, 1.758, 8.776e-02
constant factors query and depth (68%) for 0.116: 6.069e-05, 2.490, 1.243e-01
constant factors query and depth (68%) for 0.876: 2.157e-05, 0.885, 4.418e-02
constant factors query and depth (68%) for 0.766: 2.725e-05, 1.1

In [91]:
perc = 68
avg = 0.0
avg_err = 0.0
for j in range(len(avals)):
    avg += 2*np.percentile(errors1[j], perc) * num_queries[j]/len(avals)
    avg_err += np.percentile(errors1[j], perc)/len(avals)
    print(f'constant factors query and depth with known signs for {avals[j]:.3f}: {2*np.percentile(errors1[j], perc):.3e}, {2*np.percentile(errors1[j], perc) * num_queries[j]:.3f}, {2*np.percentile(errors1[j], perc) * max_single_query[j]:.3e}')
print(f'average constant factor {avg:0.3f}')
print(f'average error {avg_err:0.3e}')

constant factors query and depth with known signs for 0.400: 8.198e-05, 3.363, 1.679e-01
constant factors query and depth with known signs for 0.861: 4.407e-05, 1.808, 9.026e-02
constant factors query and depth with known signs for 0.686: 6.680e-05, 2.740, 1.368e-01
constant factors query and depth with known signs for 0.579: 7.898e-05, 3.240, 1.618e-01
constant factors query and depth with known signs for 0.225: 9.605e-05, 3.940, 1.967e-01
constant factors query and depth with known signs for 0.225: 8.623e-05, 3.537, 1.766e-01
constant factors query and depth with known signs for 0.146: 9.695e-05, 3.977, 1.986e-01
constant factors query and depth with known signs for 0.793: 5.396e-05, 2.213, 1.105e-01
constant factors query and depth with known signs for 0.581: 6.355e-05, 2.607, 1.301e-01
constant factors query and depth with known signs for 0.666: 8.926e-05, 3.661, 1.828e-01
constant factors query and depth with known signs for 0.116: 1.061e-04, 4.351, 2.172e-01
constant factors quer

In [92]:
perc = 95
avg = 0.0
avg_err = 0.0
for i in range(len(avals)):
    avg += np.percentile(errors[i], perc) * num_queries[i]/len(avals)
    avg_err += np.percentile(errors[i], perc)/len(avals)
    print(f'constant factors query and depth ({perc}%) for {avals[i]:.3f}: {np.percentile(errors[i], perc):.3e}, {np.percentile(errors[i], perc) * num_queries[i]:.3f}, {np.percentile(errors[i], perc) * max_single_query[i]:.3e}')
print(f'average constant factor {avg:0.3f}')
print(f'average error {avg_err:0.3e}')

constant factors query and depth (95%) for 0.400: 1.580e-04, 6.480, 3.235e-01
constant factors query and depth (95%) for 0.861: 6.514e-05, 2.672, 1.334e-01
constant factors query and depth (95%) for 0.686: 6.441e-05, 2.642, 1.319e-01
constant factors query and depth (95%) for 0.579: 8.406e-05, 3.448, 1.722e-01
constant factors query and depth (95%) for 0.225: 1.342e-04, 5.506, 2.749e-01
constant factors query and depth (95%) for 0.225: 2.481e-04, 10.176, 5.081e-01
constant factors query and depth (95%) for 0.146: 1.189e-04, 4.878, 2.435e-01
constant factors query and depth (95%) for 0.793: 7.055e-05, 2.894, 1.445e-01
constant factors query and depth (95%) for 0.581: 7.498e-05, 3.076, 1.536e-01
constant factors query and depth (95%) for 0.666: 7.616e-05, 3.124, 1.560e-01
constant factors query and depth (95%) for 0.116: 3.943e-04, 16.174, 8.075e-01
constant factors query and depth (95%) for 0.876: 4.195e-05, 1.721, 8.591e-02
constant factors query and depth (95%) for 0.766: 8.372e-05, 3

In [93]:
perc = 95
avg = 0.0
avg_err = 0.0
for j in range(len(avals)):
    avg += 2*np.percentile(errors1[j], perc) * num_queries[j]/len(avals)
    avg_err += np.percentile(errors1[j], perc)/len(avals)
    print(f'constant factors query and depth with known signs for {avals[j]:.3f}: {2*np.percentile(errors1[j], perc):.3e}, {2*np.percentile(errors1[j], perc) * num_queries[j]:.3f}, {2*np.percentile(errors1[j], perc) * max_single_query[j]:.3e}')
print(f'average constant factor {avg:0.3f}')
print(f'average error {avg_err:0.3e}')

constant factors query and depth with known signs for 0.400: 2.100e-04, 8.616, 4.302e-01
constant factors query and depth with known signs for 0.861: 9.106e-05, 3.735, 1.865e-01
constant factors query and depth with known signs for 0.686: 1.388e-04, 5.695, 2.843e-01
constant factors query and depth with known signs for 0.579: 1.811e-04, 7.430, 3.710e-01
constant factors query and depth with known signs for 0.225: 2.033e-04, 8.339, 4.163e-01
constant factors query and depth with known signs for 0.225: 2.309e-04, 9.473, 4.730e-01
constant factors query and depth with known signs for 0.146: 2.205e-04, 9.043, 4.515e-01
constant factors query and depth with known signs for 0.793: 1.086e-04, 4.453, 2.223e-01
constant factors query and depth with known signs for 0.581: 1.461e-04, 5.993, 2.992e-01
constant factors query and depth with known signs for 0.666: 1.458e-04, 5.981, 2.986e-01
constant factors query and depth with known signs for 0.116: 1.737e-04, 7.124, 3.557e-01
constant factors quer

In [94]:
perc = 99
avg = 0.0
avg_err = 0.0
for i in range(len(avals)):
    avg += np.percentile(errors[i], perc) * num_queries[i]/len(avals)
    avg_err += np.percentile(errors[i], perc)/len(avals)
    print(f'constant factors query and depth ({perc}%) for {avals[i]:.3f}: {np.percentile(errors[i], perc):.3e}, {np.percentile(errors[i], perc) * num_queries[i]:.3f}, {np.percentile(errors[i], perc) * max_single_query[i]:.3e}')
print(f'average constant factor {avg:0.3f}')
print(f'average error {avg_err:0.3e}')

constant factors query and depth (99%) for 0.400: 2.487e-04, 10.200, 5.092e-01
constant factors query and depth (99%) for 0.861: 1.561e-03, 64.043, 3.197e+00
constant factors query and depth (99%) for 0.686: 1.554e-04, 6.376, 3.183e-01
constant factors query and depth (99%) for 0.579: 2.493e-04, 10.226, 5.106e-01
constant factors query and depth (99%) for 0.225: 3.544e-04, 14.537, 7.258e-01
constant factors query and depth (99%) for 0.225: 4.474e-04, 18.354, 9.164e-01
constant factors query and depth (99%) for 0.146: 3.279e-04, 13.450, 6.715e-01
constant factors query and depth (99%) for 0.793: 1.003e-04, 4.116, 2.055e-01
constant factors query and depth (99%) for 0.581: 2.293e-04, 9.406, 4.696e-01
constant factors query and depth (99%) for 0.666: 1.404e-04, 5.758, 2.875e-01
constant factors query and depth (99%) for 0.116: 4.329e-04, 17.757, 8.866e-01
constant factors query and depth (99%) for 0.876: 7.571e-05, 3.105, 1.550e-01
constant factors query and depth (99%) for 0.766: 1.400e-

In [95]:
perc = 99
avg = 0.0
avg_err = 0.0
for j in range(len(avals)):
    avg += 2*np.percentile(errors1[j], perc) * num_queries[j]/len(avals)
    avg_err += np.percentile(errors1[j], perc)/len(avals)
    print(f'constant factors query and depth with known signs for {avals[j]:.3f}: {2*np.percentile(errors1[j], perc):.3e}, {2*np.percentile(errors1[j], perc) * num_queries[j]:.3f}, {2*np.percentile(errors1[j], perc) * max_single_query[j]:.3e}')
print(f'average constant factor {avg:0.3f}')
print(f'average error {avg_err:0.3e}')

constant factors query and depth with known signs for 0.400: 3.324e-04, 13.634, 6.807e-01
constant factors query and depth with known signs for 0.861: 1.010e-04, 4.145, 2.069e-01
constant factors query and depth with known signs for 0.686: 1.707e-04, 7.004, 3.497e-01
constant factors query and depth with known signs for 0.579: 2.185e-04, 8.963, 4.475e-01
constant factors query and depth with known signs for 0.225: 2.327e-04, 9.545, 4.766e-01
constant factors query and depth with known signs for 0.225: 2.594e-04, 10.640, 5.312e-01
constant factors query and depth with known signs for 0.146: 2.888e-04, 11.848, 5.915e-01
constant factors query and depth with known signs for 0.793: 1.194e-04, 4.898, 2.445e-01
constant factors query and depth with known signs for 0.581: 1.742e-04, 7.144, 3.567e-01
constant factors query and depth with known signs for 0.666: 1.574e-04, 6.456, 3.223e-01
constant factors query and depth with known signs for 0.116: 1.835e-04, 7.528, 3.759e-01
constant factors q